## Processing webscraped data

In [1]:
import graphlab as gl
from graphlab import SFrame
import pandas as pd
import numpy as np

ed_data = pd.read_csv('Edtech Recommender Webscraping Data.csv')

ed_data.head()



,product_name,grades,price,skills,subjects,product description,teacher,school,teacher rating,teacher review
0,CodeCombat,4-12,"Free, Paid",Critical Thinking,NaN,CodeCombat is an adventure game and curriculum...,Sandra A.,Meadowlark Elementary School,4.0,I think this is a great way to teach kids to c...
1,MinecraftEdu,1-12,None,Communication & Collaboration,NaN,Editor's Note: MinecraftEdu is no longer avail...,Ben B.,Kingsway Christian School,4.0,This was a great way to engage students who st...
2,Minecraft,3-12,"$26.95 for desktop, $19.99 for consoles (excep...",Communication & Collaboration,NaN,Minecraft is a sandbox game that can be adapte...,Ana P.,NaN,3.0,"Overall, I think its a fun game. Kids of all a..."
3,Quandary,2-8,Free,Character & SEL,English Language Arts,Quandary is an online ethics game that teaches...,Heather M.,Memorial Parkway Junior,4.0,The login is nice to keep track of their score...
4,BrainRush,K-12,Free,Character & SEL,English Language Arts,"At BrainRush, teachers assign students activit...",None,NaN,NaN,NaN


# Data Cleaning/Wrangling


In [2]:
ed_data['price'] = ed_data['price'].apply(lambda x: 'Paid' if x != 'Free' else 'Free')
ed_data = ed_data.rename(columns={'teacher rating': 'rating', 'product_name': 
                                  'product','product description':'description','teacher review':'review'})


ed_data.head()



,product,grades,price,skills,subjects,description,teacher,school,rating,review
0,CodeCombat,4-12,Paid,Critical Thinking,NaN,CodeCombat is an adventure game and curriculum...,Sandra A.,Meadowlark Elementary School,4.0,I think this is a great way to teach kids to c...
1,MinecraftEdu,1-12,Paid,Communication & Collaboration,NaN,Editor's Note: MinecraftEdu is no longer avail...,Ben B.,Kingsway Christian School,4.0,This was a great way to engage students who st...
2,Minecraft,3-12,Paid,Communication & Collaboration,NaN,Minecraft is a sandbox game that can be adapte...,Ana P.,NaN,3.0,"Overall, I think its a fun game. Kids of all a..."
3,Quandary,2-8,Free,Character & SEL,English Language Arts,Quandary is an online ethics game that teaches...,Heather M.,Memorial Parkway Junior,4.0,The login is nice to keep track of their score...
4,BrainRush,K-12,Free,Character & SEL,English Language Arts,"At BrainRush, teachers assign students activit...",None,NaN,NaN,NaN


In [4]:
ed_data['audience'] = np.where(ed_data['grade_size']>=6, 'Large', 'Small')

ed_data.drop(['grade_s', 'grade_l','grade_size'], axis=1, inplace=True)
ed_data.head(2)

,product,grades,price,skills,subjects,description,teacher,school,rating,review,audience
0,CodeCombat,4-12,Paid,Critical Thinking,NaN,CodeCombat is an adventure game and curriculum...,Sandra A.,Meadowlark Elementary School,4.0,I think this is a great way to teach kids to c...,Large
1,MinecraftEdu,1-12,Paid,Communication & Collaboration,NaN,Editor's Note: MinecraftEdu is no longer avail...,Ben B.,Kingsway Christian School,4.0,This was a great way to engage students who st...,Large


In [5]:
sum(ed_data['rating'].isnull())

377

In [6]:
ed_data = ed_data.loc[~(ed_data['rating'].isnull())]


In [7]:
sum(ed_data['rating'].isnull())

0

# Feature Engineering 

In [3]:
ed_data['grade_s'], ed_data['grade_l'] = ed_data['grades'].str.split('-', 1).str

ed_data['grade_s'].replace('K', 0,inplace=True)




ed_data.grade_s = pd.to_numeric(ed_data.grade_s, errors = 'coerce').fillna(0).astype(np.int64)

ed_data.grade_l = pd.to_numeric(ed_data.grade_l, errors = 'coerce').fillna(0).astype(np.int64)

ed_data['grade_size'] = ed_data['grade_l'] - ed_data['grade_s']


In [8]:
subject_to_STEM = {'English Language Arts': 'Non-STEM',
                  'Math': 'STEM',
                  'Science': 'STEM',
                  'Arts': 'Non-STEM',
                  'Social Studies': 'Non-STEM',
                  'Health & Wellness':'Non-STEM',
                  'World Language':'Non-STEM'
                  'English Language Learning',
                  'NaN':'NaN'}
ed_data['STEM'] = ed_data.subjects.map(subject_to_STEM)    

ed_data.head()

,product,grades,price,skills,subjects,description,teacher,school,rating,review,audience,STEM
0,CodeCombat,4-12,Paid,Critical Thinking,NaN,CodeCombat is an adventure game and curriculum...,Sandra A.,Meadowlark Elementary School,4.0,I think this is a great way to teach kids to c...,Large,NaN
1,MinecraftEdu,1-12,Paid,Communication & Collaboration,NaN,Editor's Note: MinecraftEdu is no longer avail...,Ben B.,Kingsway Christian School,4.0,This was a great way to engage students who st...,Large,NaN
2,Minecraft,3-12,Paid,Communication & Collaboration,NaN,Minecraft is a sandbox game that can be adapte...,Ana P.,NaN,3.0,"Overall, I think its a fun game. Kids of all a...",Large,NaN
3,Quandary,2-8,Free,Character & SEL,English Language Arts,Quandary is an online ethics game that teaches...,Heather M.,Memorial Parkway Junior,4.0,The login is nice to keep track of their score...,Large,Non-STEM
5,Spent,7-12,Free,Character & SEL,Social Studies,"Spent is sort of like a jazzed-up, text-based,...",Elizabeth H.,Arcadia High School,4.0,One of the parts that I like best is that afte...,Small,Non-STEM


In [9]:
skills_to_skill_type = {'Communication & Collaboration': 'Soft',
                        'Critical Thinking': 'Hard',
                        'Character & SEL': 'Soft',
                        'Creativity': 'Hard',
                        'None': 'NaN',
                        'College & Career Prep': 'Hard'}

ed_data['skill_type'] = ed_data.skills.map(skills_to_skill_type)

ed_data.head(2)

,product,grades,price,skills,subjects,description,teacher,school,rating,review,audience,STEM,skill_type
0,CodeCombat,4-12,Paid,Critical Thinking,NaN,CodeCombat is an adventure game and curriculum...,Sandra A.,Meadowlark Elementary School,4.0,I think this is a great way to teach kids to c...,Large,NaN,Hard
1,MinecraftEdu,1-12,Paid,Communication & Collaboration,NaN,Editor's Note: MinecraftEdu is no longer avail...,Ben B.,Kingsway Christian School,4.0,This was a great way to engage students who st...,Large,NaN,Soft


In [10]:
ed_data['school_type'] = pd.np.where(ed_data.school.str.contains("Primary"), "Elementary",
                   pd.np.where(ed_data.school.str.contains("Elementary"), "Elementary",
                    pd.np.where(ed_data.school.str.contains("Junior"), "Middle",
                    pd.np.where(ed_data.school.str.contains("Middle"), "Middle",
                    pd.np.where(ed_data.school.str.contains("Intermediate"), "Middle", 
                    pd.np.where(ed_data.school.str.contains("High"), "High", 
                    pd.np.where(ed_data.school.str.contains("Academy"), "High", 
                    pd.np.where(ed_data.school.str.contains("County"), "High", 
                    pd.np.where(ed_data.school.str.contains("University"), "College", 
                    pd.np.where(ed_data.school.str.contains("International"), "International", 
                   pd.np.where(ed_data.school.str.contains("NaN"), "Missing", 'Missing')))))))))))
    

ed_data.head()                     


,product,grades,price,skills,subjects,description,teacher,school,rating,review,audience,STEM,skill_type,school_type
0,CodeCombat,4-12,Paid,Critical Thinking,NaN,CodeCombat is an adventure game and curriculum...,Sandra A.,Meadowlark Elementary School,4.0,I think this is a great way to teach kids to c...,Large,NaN,Hard,Elementary
1,MinecraftEdu,1-12,Paid,Communication & Collaboration,NaN,Editor's Note: MinecraftEdu is no longer avail...,Ben B.,Kingsway Christian School,4.0,This was a great way to engage students who st...,Large,NaN,Soft,Missing
2,Minecraft,3-12,Paid,Communication & Collaboration,NaN,Minecraft is a sandbox game that can be adapte...,Ana P.,NaN,3.0,"Overall, I think its a fun game. Kids of all a...",Large,NaN,Soft,Elementary
3,Quandary,2-8,Free,Character & SEL,English Language Arts,Quandary is an online ethics game that teaches...,Heather M.,Memorial Parkway Junior,4.0,The login is nice to keep track of their score...,Large,Non-STEM,Soft,Middle
5,Spent,7-12,Free,Character & SEL,Social Studies,"Spent is sort of like a jazzed-up, text-based,...",Elizabeth H.,Arcadia High School,4.0,One of the parts that I like best is that afte...,Small,Non-STEM,Soft,High


In [13]:
ed_data = ed_data.astype(str)
data_sf = gl.SFrame(ed_data)
user_info = data_sf[['teacher', 'product', 'school', 'school_type']].copy()
item_info = data_sf[['product', 'price', 'skills', 'subjects','audience','STEM','skill_type']].copy
data_sf['rating'] = data_sf['rating'].astype(int)


# Tfidf 

In [14]:
data_sf['word_count'] = gl.text_analytics.count_words(data_sf['description'])

data_sf['word_count'].dict_trim_by_keys(gl.text_analytics.stopwords(), True)

tfidf = gl.text_analytics.tf_idf(data_sf['word_count'])
data_sf['tfidf'] = tfidf

data_sf.head()

product,grades,price,skills,subjects,description
CodeCombat,4-12,Paid,Critical Thinking,nan,CodeCombat is anadventure game and ...
MinecraftEdu,1-12,Paid,Communication &Collaboration ...,nan,Editor's Note:MinecraftEdu is no lo ...
Minecraft,3-12,Paid,Communication &Collaboration ...,nan,Minecraft is a sandboxgame that can be adapted ...
Quandary,2-8,Free,Character & SEL,English Language Arts,Quandary is an onlineethics game that teaches ...
Spent,7-12,Free,Character & SEL,Social Studies,"Spent is sort of like ajazzed-up, text-based, ..."
Prodigy,1-8,Paid,Critical Thinking,Math,"In Prodigy, an adaptivemath-practice game se ..."
Nitro Type,3-8,Paid,nan,English Language Arts,Nitro Type is amultiplayer car racing ...
The Time Tribe,4-6,Free,Character & SEL,Social Studies,Interested in teachinghistory by letting your ...
Zondle,1-5,Paid,Creativity,nan,"Editor's Note:Zondle closed on May 1, ..."
Flowlab,7-12,Paid,Creativity,Arts,Flowlab is a game-makingtool that’s targeted ...


In [15]:
data_pd = data_sf.to_dataframe()

In [20]:
keyword_column = []

def make_keywords(data_pd):
    i = 0
    for i in range(994):
        cell = data_pd.iloc[i,15]
        keywords = {k for k, v in cell.iteritems() if v > 6}
        keyword_column.append(keywords)
        i = i+1




In [21]:
make_keywords(data_pd)

In [22]:
data_pd['keywords'] = keyword_column

data_pd.tail()

,product,grades,price,skills,subjects,description,teacher,school,rating,review,audience,STEM,skill_type,school_type,word_count,tfidf,keywords
989,Math Pack Flash Cards,4-8,Paid,Character & SEL,Math,Math Pack Flash Cards is a series of well-orga...,Michelle K.,South Western School,3,<p>Math Pack Flash Cards App is a comprehensiv...,Small,STEM,Soft,Missing,"{u'and': 2, u'simple': 2, u'sweet': 1, u'into'...","{u'and': 0.130955858759, u'simple': 4.96579319...","{eighth-grade-level, tucked, all), one-digit, ..."
990,Mystery Word Town - Sight Word Spelling,1-4,Paid,Critical Thinking,English Language Arts,Welcome to Mystery Word Town - Sight Word Spel...,Jennifer K.,nan,4,The Mystery of the app is that students find ...,Small,Non-STEM,Hard,Elementary,"{u'and': 3, u'robbers': 1, u'set': 1, u'have':...","{u'and': 0.196433788139, u'robbers': 6.9017372...","{use, or, outlaws., robbers, are three, gold,,..."
991,AB Math - fun game for kids and grownups,1-6,Paid,Critical Thinking,Math,AB Math is a solid and well-organized math app...,Hannah B.,Orchard Lane Elementary,4,AB Math is a very simple and easy-to-use app. ...,Small,STEM,Hard,Elementary,"{u'and': 5, u'it.': 1, u'""bubbles""': 1, u'ab':...","{u'and': 0.327389646898, u'it.': 3.60590034065...","{numerals, ""bubbles"", ab, ordering., kids'quiz..."
992,Photo Grid - Collage Maker,3-8,Free,Communication & Collaboration,Arts,Photo Grid – Collage Maker is a photo app that...,Kristy (GAE) L.,P.S. 200 Benson,4,Photo Grid Collage Maker is a great tool for s...,Small,Non-STEM,Soft,Missing,"{u'photo': 2, u'won't': 1, u'find': 1, u'backg...","{u'photo': 6.87200260771, u'won't': 4.95582705...","{(ios, background;, layouts. from, grids, phot..."
993,IXL Math and English,Pre-K-12,Paid,Character & SEL,English Language Arts,IXL - Math and English is a handy tool that ma...,Sarah W.,nan,5,I love this product! This is a product where t...,Small,Non-STEM,Soft,Elementary,"{u'and': 6, u'math,': 1, u'score,': 1, u'then,...","{u'and': 0.392867576277, u'math,': 3.723683376...","{grade-appropriate, ixl, -, area and, praise, ..."


In [26]:
ed_data_sf = gl.SFrame(data_pd)

ed_data_sf.head()

product,grades,price,skills,subjects,description
CodeCombat,4-12,Paid,Critical Thinking,nan,CodeCombat is anadventure game and ...
MinecraftEdu,1-12,Paid,Communication &Collaboration ...,nan,Editor's Note:MinecraftEdu is no lo ...
Minecraft,3-12,Paid,Communication &Collaboration ...,nan,Minecraft is a sandboxgame that can be adapted ...
Quandary,2-8,Free,Character & SEL,English Language Arts,Quandary is an onlineethics game that teaches ...
Spent,7-12,Free,Character & SEL,Social Studies,"Spent is sort of like ajazzed-up, text-based, ..."
Prodigy,1-8,Paid,Critical Thinking,Math,"In Prodigy, an adaptivemath-practice game se ..."
Nitro Type,3-8,Paid,nan,English Language Arts,Nitro Type is amultiplayer car racing ...
The Time Tribe,4-6,Free,Character & SEL,Social Studies,Interested in teachinghistory by letting your ...
Zondle,1-5,Paid,Creativity,nan,"Editor's Note:Zondle closed on May 1, ..."
Flowlab,7-12,Paid,Creativity,Arts,Flowlab is a game-makingtool that’s targeted ...


In [27]:
train, test = gl.recommender.util.random_split_by_user(ed_data_sf,
                                                              user_id = 'teacher',
                                                              item_id = 'product',
                                                              item_test_proportion = 0.2,
                                                              random_seed = 100)

In [36]:
weights_dic = {'grades':.01,
           'price':.02,
           'skills': .15,
           'subjects':.15,
           'description': .15,
           'teacher': .01,
           'school':.01,
           'rating':.20,
           'review':.05,
           'audience':.02,
           'STEM':.02,
           'skill_type':.02,
           'school_type':.02,
           'word_count':.01,
           'tfidf':.01,
           'keywords':.15}

In [37]:
 rec_model = gl.recommender.item_content_recommender.create(train, item_id = 'product',
                                                           observation_data = train, 
                                                            user_id = 'teacher',
                                                            target = 'rating', 
                                                           weights = weights_dic)


('Applying transform:\n', Class             : AutoVectorizer

Model Fields
------------
Features          : ['grades', 'price', 'skills', 'subjects', 'description', 'teacher', 'school', 'rating', 'review', 'audience', 'STEM', 'skill_type', 'school_type', 'word_count', 'tfidf', 'keywords']
Excluded Features : ['product']

Column       Type  Interpretation  Transforms                         Output Type
-----------  ----  --------------  ---------------------------------  -----------
grades       str   categorical     None                               str        
price        str   categorical     None                               str        
skills       str   categorical     None                               str        
subjects     str   categorical     None                               str        
description  str   long_text       2-Word NGram Counts -> TFIDF       dict       
teacher      str   short_text      3-Character NGram Counts -> TFIDF  dict       
school       str   sh

Recsys training: model = item_content_recommender

Defaulting to brute force instead of ball tree because there are multiple distance components.


Starting brute force nearest neighbors model training.

Starting pairwise querying.

...

Data has 789 observations with 555 users and 783 items.

Data prepared in: 0.201769s

Loading user-provided nearest items.

Generating candidate set for working with new users.

Finished training in 0.013174s

# Final Recommendations for list of teachers


In [38]:
rec_model.recommend(users = data_sf['teacher'], k = 5).print_rows(num_rows =25)

+--------------+-------------------------------+---------------+------+
|   teacher    |            product            |     score     | rank |
+--------------+-------------------------------+---------------+------+
|  Sandra A.   |            Quia Web           | 2.51963734627 |  1   |
|  Sandra A.   |           True Color          | 2.40164542198 |  2   |
|  Sandra A.   |           DuckDuckGo          | 2.33035349846 |  3   |
|  Sandra A.   |            Declara            | 2.28253245354 |  4   |
|  Sandra A.   |             Gooru             | 2.25716280937 |  5   |
|    Ben B.    |            Popplet            | 4.21755862236 |  1   |
|    Ben B.    |     Mozilla Popcorn Maker     |  2.9020960331 |  2   |
|    Ben B.    |     Movenote for Education    | 2.34681868553 |  3   |
|    Ben B.    |        Thomas Was Alone       | 2.34628796577 |  4   |
|    Ben B.    |  Seesaw: The Learning Journal | 2.33715844154 |  5   |
|    Ana P.    |          Code Monkey          | 3.16336005926 |

In [39]:
rec_model.get_similar_items(items = ['CodeCombat', 'Quandary', 'Spent','Prodigy'], k = 3)

product,similar,score,rank
CodeCombat,Quia Web,0.629909336567,1
CodeCombat,True Color,0.600411355495,2
CodeCombat,DuckDuckGo,0.582588374615,3
Quandary,Lumosity,0.599122822285,1
Quandary,Constitute,0.595634758472,2
Quandary,Win the White House,0.595395982265,3
Prodigy,Code Monkey,0.632672011852,1
Prodigy,Slice Fractions,0.614138126373,2
Prodigy,Pick-a-Path,0.604565322399,3


In [58]:
rec_model.evaluate(test, metric = 'rmse')

('\nOverall RMSE: ', 1.0524355187740462)

Per User RMSE (best)
+----------+-------+------------------+
| teacher  | count |       rmse       |
+----------+-------+------------------+
| Laura E. |   1   | 0.00148963372616 |
+----------+-------+------------------+
[1 rows x 3 columns]


Per User RMSE (worst)
+----------+-------+---------------+
| teacher  | count |      rmse     |
+----------+-------+---------------+
| Aaron S. |   1   | 2.27695875199 |
+----------+-------+---------------+
[1 rows x 3 columns]


Per Item RMSE (best)
+------------+-------+------------------+
|  product   | count |       rmse       |
+------------+-------+------------------+
| Xtranormal |   1   | 0.00148963372616 |
+------------+-------+------------------+
[1 rows x 3 columns]


Per Item RMSE (worst)
+----------------+-------+---------------+
|    product     | count |      rmse     |
+----------------+-------+---------------+
| Natural Reader |   1   | 2.27695875199 |
+----------------+-------+----------

{'rmse_by_item': Columns:
 	product	str
 	count	int
 	rmse	float
 
 Rows: 205
 
 Data:
 +--------------------------------+-------+----------------+
 |            product             | count |      rmse      |
 +--------------------------------+-------+----------------+
 |            Fraxinus            |   1   | 1.03490353419  |
 |          My PlayHome           |   1   | 0.509624563189 |
 |     DreamBox Learning Math     |   1   | 1.96445744534  |
 | Lightbot - Programming Puzzles |   1   | 0.31917817335  |
 |        SesameStreet.org        |   1   | 0.87079345345  |
 |          Cell Command          |   1   | 2.14395805917  |
 |       Mosa Mack Science        |   1   | 1.83380488263  |
 |         LyricsTraining         |   1   | 1.28898508844  |
 |           Goodreads            |   1   | 0.953117731059 |
 | Wonder for Dash and Dot Robots |   1   | 1.50273150628  |
 +--------------------------------+-------+----------------+
 [205 rows x 3 columns]
 Note: Only the head of the SFrame 